MÉTRICA:

*   Cobertura de los programas académicos de tecnología en Medellín.

DATOS:

*   Número de estudiantes y número de graduados por programa académico
*   Año
*   Cobertura

PREGUNTAS:

*   ¿Cuál es el número de estudiantes por IES por año?
*   ¿Cuál es el número de estudiantes por nivel de formación por año?
*   ¿Cuál es el número de estudiantes por Núcleo Básico del Conocimiento (NBC) por año?










In [1]:
!git clone "http://github.com/dcardonaa21/WTM_Medellin_Pilot.git"


Cloning into 'WTM_Medellin_Pilot'...
remote: Enumerating objects: 15, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 93 (delta 6), reused 0 (delta 0), pack-reused 78
Unpacking objects: 100% (93/93), done.


In [2]:
!pip install pyxlsb

In [3]:
import sys 
sys.path.insert(0, "/content/WTM_Medellin_Pilot/STEM_Studio")
sys.path.insert(1, "/content/WTM_Medellin_Pilot/Piloto_Medellin")
import stem_library as sl
import libreria_piloto_medellin as lpm
import pandas as pd
import pathlib
import numpy as np

In [4]:
def calligraphy_unification(names_ls):
    """
    Replaces every spanish accentuated vowel by a simple vowel, takes "*" out from the string and turns the strings into uppercase strings.
    Parameters:
    names_ls: List of strings.
    Returns:
    names_dict: A dictionary with the original strings as keys and uppercase clean strings as values.
    """
    invalid_vowels_dict = {"á":"a", "é":"e", "í":"i", "ó":"o", "ú":"u", "ü":"u", "Á":"A", "É":"E", "Í":"I", "Ó":"O", "Ú":"U", "Ü":"U", "*":"", ",":"", "-":""}
    characters_ls = []
    new_characters_ls = []
    names_dict = dict()
    new_name = str()
    for name in names_ls:
        characters_ls = list(name)
        for character in characters_ls:
            if character in invalid_vowels_dict.keys():
                new_char = invalid_vowels_dict[character]
                new_characters_ls.append(new_char)
            else:
                new_characters_ls.append(character)
        new_name = ("".join(new_characters_ls)).upper()
        names_dict[name] = new_name
        new_name = str()
        new_characters_ls= []
        characters_ls = []
    return names_dict

Leer archivo de excel XLSB:

In [5]:
matriculados_hasta_2013_dict = pd.read_excel("https://storage.googleapis.com/piloto_medellin/excel/MATRICULADOS-HASTA-2013.xlsb", sheet_name=None, engine="pyxlsb")

Ver pestañas del archivo de excel:

In [6]:
matriculados_hasta_2013_dict.keys()

dict_keys(['Matriculados 2000-2013'])

Definir dataframe que contenga dataset: 

In [7]:
matriculados_hasta_2013_df = matriculados_hasta_2013_dict['Matriculados 2000-2013']
matriculados_hasta_2013_df.head(10)

,MATRICULADOS EN EDUCACIÓN SUPERIOR - COLOMBIA 2000 - 2013,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,...,Unnamed: 63,Unnamed: 64,Unnamed: 65,Unnamed: 66,Unnamed: 67,Unnamed: 68,Unnamed: 69,Unnamed: 70,Unnamed: 71,Unnamed: 72,Unnamed: 73,Unnamed: 74,Unnamed: 75,Unnamed: 76,Unnamed: 77,Unnamed: 78,Unnamed: 79,Unnamed: 80,Unnamed: 81,Unnamed: 82,Unnamed: 83,Unnamed: 84,Unnamed: 85,Unnamed: 86,Unnamed: 87,Unnamed: 88,Unnamed: 89,Unnamed: 90,Unnamed: 91,Unnamed: 92,Unnamed: 93,Unnamed: 94,Unnamed: 95,Unnamed: 96,Unnamed: 97,Unnamed: 98,Unnamed: 99,Unnamed: 100,Unnamed: 101,Unnamed: 102
0,Fecha de corte de la información: junio 30 de ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Fuente: MEN - SNIES,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"Cifras SENA 2011, 2012 y 2013 ajustadas a Dici...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,La información suministrada corresponde a lo r...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Código de \nla Institución,Institución de Educación Superior (IES),Principal\n o\nSeccional,Sector IES,Carácter IES,Código del \ndepartamento\n(IES),Departamento de \ndomicilio de la IES,Código del \nMunicipio\n(IES),Municipio de\ndomicilio de la IES,Código \nSNIES del\nprograma,Programa Académico,Nivel de Formación,Metodología \ndel programa,Área de Conocimiento,Núcleo Básico del Conocimiento (NBC),Código del \nDepartamento\n(Programa),Departamento de oferta del programa,Código del \nMunicipio\n(Programa),Municipio de oferta del programa,Hombre 2000-1,Mujer 2000-1,Total 2000-1,Hombre 2000-2,Mujer 2000-2,Total 2000-2,Hombre 2001-1,Mujer 2001-1,Total 2001-1,Hombre 2001-2,Mujer 2001-2,Total 2001-2,Hombre 2002-1,Mujer 2002-1,Total 2002-1,Hombre 2002-2,Mujer 2002-2,Total 2002-2,Hombre 2003-1,Mujer 2003-1,Total 2003-1,...,Total 2007-1,Hombre 2007-2,Mujer 2007-2,Total 2007-2,Hombre 2008-1,Mujer 2008-1,Total 2008-1,Hombre 2008-2,Mujer 2008-2,Total 2008-2,Hombre 2009-1,Mujer 2009-1,Total 2009-1,Hombre 2009-2,Mujer 2009-2,Total 2009-2,Hombre 2010-1,Mujer 2010-1,Total 2010-1,Hombre 2010-2,Mujer 2010-2,Total 2010-2,Hombre 2011-1,Mujer 2011-1,Total 2011-1,Hombre 2011-2,Mujer 2011-2,Total 2011-2,Hombr

Definir diccionario con los nombres de las columnas que se quieren cambiar:

In [8]:
rename_columns_dict = {
 "Unnamed: 1": 'INSTITUCION DE EDUCACION SUPERIOR (IES)',
 "Unnamed: 11": 'NIVEL DE FORMACION',
 "Unnamed: 14": 'NUCLEO BASICO DEL CONOCIMIENTO (NBC)'
}

Limpiar filas vacías:

In [9]:
matriculados_hasta_2013_df.drop(matriculados_hasta_2013_df.index[0:6], inplace=True)


Verificar:

In [10]:
matriculados_hasta_2013_df.head()


,MATRICULADOS EN EDUCACIÓN SUPERIOR - COLOMBIA 2000 - 2013,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,...,Unnamed: 63,Unnamed: 64,Unnamed: 65,Unnamed: 66,Unnamed: 67,Unnamed: 68,Unnamed: 69,Unnamed: 70,Unnamed: 71,Unnamed: 72,Unnamed: 73,Unnamed: 74,Unnamed: 75,Unnamed: 76,Unnamed: 77,Unnamed: 78,Unnamed: 79,Unnamed: 80,Unnamed: 81,Unnamed: 82,Unnamed: 83,Unnamed: 84,Unnamed: 85,Unnamed: 86,Unnamed: 87,Unnamed: 88,Unnamed: 89,Unnamed: 90,Unnamed: 91,Unnamed: 92,Unnamed: 93,Unnamed: 94,Unnamed: 95,Unnamed: 96,Unnamed: 97,Unnamed: 98,Unnamed: 99,Unnamed: 100,Unnamed: 101,Unnamed: 102
6,1101,UNIVERSIDAD NACIONAL DE COLOMBIA,PRINCIPAL,OFICIAL,UNIVERSIDAD,11,BOGOTA D.C.,11001,BOGOTA D.C.,1,INGENIERIA AGRONOMICA,UNIVERSITARIA,PRESENCIAL,"INGENIERIA, ARQUITECTURA, URBANISMO Y AFINES","INGENIERIA AGRONOMICA, PECUARIA Y AFINES",11,BOGOTA D.C.,11001,BOGOTA D.C.,494,252,746,531,261,792,254,530,784,251,504,755,498,260,758,481,250,731,467,250,717,...,658,440,211,651,416,205,621,416,205,621,382,203,585,396,201,597,394,187,581,397,192,589,402,174,576,412,193,605,448,208,656,460,220,680,474,208,682,447,211,658
7,1101,UNIVERSIDAD NACIONAL DE COLOMBIA,PRINCIPAL,OFICIAL,UNIVERSIDAD,11,BOGOTA D.C.,11001,BOGOTA D.C.,2,MEDICINA VETERINARIA,UNIVERSITARIA,PRESENCIAL,"AGRONOMIA, VETERINARIA Y AFINES",MEDICINA VETERINARIA,11,BOGOTA D.C.,11001,BOGOTA D.C.,352,239,591,334,242,576,259,351,610,260,351,611,342,260,602,333,273,606,293,327,620,...,536,252,256,508,243,251,494,245,266,511,255,258,513,251,259,510,257,260,517,254,266,520,261,281,542,261,286,547,256,277,533,261,284,545,264,284,548,272,278,550
8,1101,UNIVERSIDAD NACIONAL DE COLOMBIA,PRINCIPAL,OFICIAL,UNIVERSIDAD,11,BOGOTA D.C.,11001,BOGOTA D.C.,3,ZOOTECNIA,UNIVERSITARIA,PRESENCIAL,"AGRONOMIA, VETERINARIA Y AFINES",ZOOTECNIA,11,BOGOTA D.C.,11001,BOGOTA D.C.,327,162,489,313,149,462,152,319,471,154,316,470,326,156,482,328,159,487,354,147,501,...,476,301,180,481,318,184,502,314,180,494,313,170,483,306,165,471,299,171,470,300,165,465,319,162,481,298,139,437,309,147,456,308,153,461,288,155,443,272,149,421
9,1101,UNIVERSIDAD NACIONAL DE COLOMBIA,PRINCIPAL,OFICIAL,UNIVERSIDAD,11,BOGOTA D.C.,11001,BOGOTA D.C.,4,DISEÑO GRAFICO,UNIVERSITARIA,PRESENCIAL,BELLAS ARTES,DISEÑO,11,BOGOTA D.C.,11001,BOGOTA D.C.,198,65,263,202,68,270,58,201,259,53,208,261,193,50,243,199,40,239,197,36,233,...,262,192,71,263,197,73,270,197,76,273,200,74,274,202,82,284,200,82,282,192,87,279,203,89,292,207,97,304,205,93,298,191,89,280,202,87,289,193,91,284
10,1101,UNIVERSIDAD NACIONAL DE COLOMBIA,PRINCIPAL,OFICIAL,UNIVERSIDAD,11,BOGOTA D.C.,11001,BOGOTA D.C.,5,DISEÑO INDUSTRIAL,UNIVERSITARIA,PRESENCIAL,BELLAS ARTES,DISEÑO,11,BOGOTA D.C.,11001,BOGOTA D.C.,316,161,477,354,169,523,170,357,527,170,381,551,373,166,539,359,162,521,375,159,534,...,483,296,161,457,296,165,461,290,167,457,280,162,442,277,162,439,278,161,439,286,152,438,290,157,447,296,154,450,311,169,480,305,171,476,309,174,483,304,171,475


In [11]:
matriculados_hasta_2013_df = matriculados_hasta_2013_df.loc[matriculados_hasta_2013_df["Unnamed: 8"] == "MEDELLIN", :]

Fraccionar el dataframe en 2 para trabajar con las columnas de los semestres por separado. Filtrar columnas de interés sin los semestres:

In [12]:
filtered1_df = matriculados_hasta_2013_df.filter(["Unnamed: 1", "Unnamed: 11", "Unnamed: 14"])

Verificar:

In [13]:
filtered1_df.head()

,Unnamed: 1,Unnamed: 11,Unnamed: 14
289,UNIVERSIDAD NACIONAL DE COLOMBIA,UNIVERSITARIA,"INGENIERIA AGRONOMICA, PECUARIA Y AFINES"
290,UNIVERSIDAD NACIONAL DE COLOMBIA,UNIVERSITARIA,ECONOMIA
291,UNIVERSIDAD NACIONAL DE COLOMBIA,UNIVERSITARIA,"GEOGRAFIA, HISTORIA"
292,UNIVERSIDAD NACIONAL DE COLOMBIA,UNIVERSITARIA,"INGENIERIA AGRICOLA, FORESTAL Y AFINES"
293,UNIVERSIDAD NACIONAL DE COLOMBIA,UNIVERSITARIA,INGENIERIA CIVIL Y AFINES


Renombrar las columnas del dataframe filtrado:

In [14]:
filtered1_df.rename(columns=rename_columns_dict, inplace=True )

Verificar:

In [15]:
filtered1_df.head()

,INSTITUCION DE EDUCACION SUPERIOR (IES),NIVEL DE FORMACION,NUCLEO BASICO DEL CONOCIMIENTO (NBC)
289,UNIVERSIDAD NACIONAL DE COLOMBIA,UNIVERSITARIA,"INGENIERIA AGRONOMICA, PECUARIA Y AFINES"
290,UNIVERSIDAD NACIONAL DE COLOMBIA,UNIVERSITARIA,ECONOMIA
291,UNIVERSIDAD NACIONAL DE COLOMBIA,UNIVERSITARIA,"GEOGRAFIA, HISTORIA"
292,UNIVERSIDAD NACIONAL DE COLOMBIA,UNIVERSITARIA,"INGENIERIA AGRICOLA, FORESTAL Y AFINES"
293,UNIVERSIDAD NACIONAL DE COLOMBIA,UNIVERSITARIA,INGENIERIA CIVIL Y AFINES


Cambiar índices:

In [16]:
filtered1_df.index = list(range(0, len(filtered1_df.iloc[:, 0])))
filtered1_df

,INSTITUCION DE EDUCACION SUPERIOR (IES),NIVEL DE FORMACION,NUCLEO BASICO DEL CONOCIMIENTO (NBC)
0,UNIVERSIDAD NACIONAL DE COLOMBIA,UNIVERSITARIA,"INGENIERIA AGRONOMICA, PECUARIA Y AFINES"
1,UNIVERSIDAD NACIONAL DE COLOMBIA,UNIVERSITARIA,ECONOMIA
2,UNIVERSIDAD NACIONAL DE COLOMBIA,UNIVERSITARIA,"GEOGRAFIA, HISTORIA"
3,UNIVERSIDAD NACIONAL DE COLOMBIA,UNIVERSITARIA,"INGENIERIA AGRICOLA, FORESTAL Y AFINES"
4,UNIVERSIDAD NACIONAL DE COLOMBIA,UNIVERSITARIA,INGENIERIA CIVIL Y AFINES
...,...,...,...
3185,CORPORACION UNIVERSITARIA DE CIENCIA Y TECNOLO...,UNIVERSITARIA,DERECHO Y AFINES
3186,SERVICIO NACIONAL DE APRENDIZAJE-SENA-,TECNICA PROFESIONAL,DERECHO Y AFINES
3187,SERVICIO NACIONAL DE APRENDIZAJE-SENA-,TECNOLOGICA,"INGENIERIA ELECTRONICA, TELECOMUNICACIONES Y A..."
3188,SERVICIO NACIONAL DE APRENDIZAJE-SENA-,TECNOLOGICA,INGENIERIA MECANICA Y AFINES


Crear dataframe con columnas de los semestres:

In [17]:
matriculados_hasta_2013_df.loc[:, "Unnamed: 21":"Unnamed: 102"]

,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51,Unnamed: 52,Unnamed: 53,Unnamed: 54,Unnamed: 55,Unnamed: 56,Unnamed: 57,Unnamed: 58,Unnamed: 59,Unnamed: 60,...,Unnamed: 63,Unnamed: 64,Unnamed: 65,Unnamed: 66,Unnamed: 67,Unnamed: 68,Unnamed: 69,Unnamed: 70,Unnamed: 71,Unnamed: 72,Unnamed: 73,Unnamed: 74,Unnamed: 75,Unnamed: 76,Unnamed: 77,Unnamed: 78,Unnamed: 79,Unnamed: 80,Unnamed: 81,Unnamed: 82,Unnamed: 83,Unnamed: 84,Unnamed: 85,Unnamed: 86,Unnamed: 87,Unnamed: 88,Unnamed: 89,Unnamed: 90,Unnamed: 91,Unnamed: 92,Unnamed: 93,Unnamed: 94,Unnamed: 95,Unnamed: 96,Unnamed: 97,Unnamed: 98,Unnamed: 99,Unnamed: 100,Unnamed: 101,Unnamed: 102
289,242,145,88,233,91,147,238,75,138,213,142,79,221,130,82,212,171,98,269,184,123,307,195,134,329,216,139,355,241,151,392,219,163,382,202,166,368,229,159,388,...,372,215,165,380,198,163,361,235,187,422,213,176,389,198,168,366,224,167,391,198,156,354,228,162,390,200,181,381,215,186,401,196,182,378,177,155,332,169,139,308
290,371,160,220,380,218,156,374,185,138,323,136,193,329,142,204,346,164,212,376,154,212,366,153,201,354,163,198,361,169,207,376,171,199,370,153,180,333,163,188,351,...,339,139,159,298,145,154,299,154,162,316,153,144,297,163,142,305,174,150,324,158,131,289,221,145,366,218,135,353,187,136,323,189,121,310,189,127,316,188,111,299
291,228,140,115,255,120,145,265,116,140,256,142,117,259,148,121,269,150,126,276,152,131,283,153,137,290,162,127,289,166,153,319,165,138,303,169,135,304,166,132,298,...,307,158,126,284,169,127,296,168,125,293,137,105,242,150,92,242,151,102,253,155,112,267,169,116,285,157,122,279,149,106,255,151,102,253,143,99,242,134,92,226
292,147,96,53,149,45,95,140,46,86,132,76,37,113,82,40,122,95,46,141,97,49,146,113,67,180,122,80,202,144,98,242,128,91,219,123,94,217,139,88,227,...,228,122,80,202,141,86,227,178,99,277,154,91,245,144,99,243,159,109,268,126,105,231,151,117,268,154,129,283,165,135,300,148,124,272,158,112,270,155,111,266
293,852,624,270,894,281,614,895,286,602,888,586,272,858,582,281,863,623,277,900,592,276,868,553,247,800,559,231,790,554,252,806,530,235,765,533,214,747,552,220,772,...,761,542,212,754,548,209,757,547,212,759,537,210,747,521,218,739,534,211,745,522,226,748,605,241,846,614,254,868,590,244,834,587,230,817,578,224,802,596,230,826
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26803,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,19,11,30,23,15,38
38330,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,51,65,116,51,65,116,74,123,197,74,123,197,42,82,124,42,82,124,0,0,0,0,0,0,...,42,10,32,42,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
38331,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,47,0,47,47,0,47,22,0,22,22,0,22,60,2,62,60,2,62,38,2,40,38,2,40,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
38332,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,67,2,69,67,2,69,107,2,109,107,2,109,224,6,230,224,6,230,254,13,267,254,13,267,...,185,177,8,185,111,8,119,111,8,119,42,8,50,42,8,50,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


#Registros totales: 38329

Convertir el dataframe a numpy array para hacer un filtrado por medio de slicing:

In [18]:
anio_array = np.array(matriculados_hasta_2013_df.loc[:, "Unnamed: 21":"Unnamed: 102"])
anio_array

array([[242, 145, 88, ..., 169, 139, 308],
       [371, 160, 220, ..., 188, 111, 299],
       [228, 140, 115, ..., 134, 92, 226],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=object)

In [19]:
anio_array.shape

(3190, 82)

Hacer slicing para seleccionar únicamente las columnas de los matriculados totales por semestre:

In [20]:
sliced_anio_array = anio_array[:, ::3]

In [21]:
sliced_anio_array.shape

(3190, 28)

Volver a convertir a dataframe para nombrar columnas y concatenar con filtered1_df:

In [22]:
semestres_df = pd.DataFrame(sliced_anio_array)
semestres_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27
0,242,233,238,213,221,212,269,307,329,355,392,382,368,388,372,380,361,422,389,366,391,354,390,381,401,378,332,308
1,371,380,374,323,329,346,376,366,354,361,376,370,333,351,339,298,299,316,297,305,324,289,366,353,323,310,316,299
2,228,255,265,256,259,269,276,283,290,289,319,303,304,298,307,284,296,293,242,242,253,267,285,279,255,253,242,226
3,147,149,140,132,113,122,141,146,180,202,242,219,217,227,228,202,227,277,245,243,268,231,268,283,300,272,270,266
4,852,894,895,888,858,863,900,868,800,790,806,765,747,772,761,754,757,759,747,739,745,748,846,868,834,817,802,826
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3185,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,30,38
3186,0,0,0,0,0,0,116,116,197,197,124,124,0,0,42,42,0,0,0,0,0,0,0,0,0,0,0,0
3187,0,0,0,0,0,0,47,47,22,22,62,62,40,40,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3188,0,0,0,0,0,0,69,69,109,109,230,230,267,267,185,185,119,119,50,50,0,0,0,0,0,0,0,0


In [23]:
len(semestres_df.columns)

28

Creamos nuevas columnas con la suma de matriculados por año:

In [24]:
for i in range(0, 14):
  año = str(i+2000)
  semestres_df[año] = semestres_df.iloc[:, 2*i] + semestres_df.iloc[:, (2*i)+1]

semestres_df.head()


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013
0,242,233,238,213,221,212,269,307,329,355,392,382,368,388,372,380,361,422,389,366,391,354,390,381,401,378,332,308,475,451,433,576,684,774,756,752,783,755,745,771,779,640
1,371,380,374,323,329,346,376,366,354,361,376,370,333,351,339,298,299,316,297,305,324,289,366,353,323,310,316,299,751,697,675,742,715,746,684,637,615,602,613,719,633,615
2,228,255,265,256,259,269,276,283,290,289,319,303,304,298,307,284,296,293,242,242,253,267,285,279,255,253,242,226,483,521,528,559,579,622,602,591,589,484,520,564,508,468
3,147,149,140,132,113,122,141,146,180,202,242,219,217,227,228,202,227,277,245,243,268,231,268,283,300,272,270,266,296,272,235,287,382,461,444,430,504,488,499,551,572,536
4,852,894,895,888,858,863,900,868,800,790,806,765,747,772,761,754,757,759,747,739,745,748,846,868,834,817,802,826,1746,1783,1721,1768,1590,1571,1519,1515,1516,1486,1493,1714,1651,1628


In [25]:
semestres_df.shape

(3190, 42)

Creamos nuevo dataframe que contenga la suma de matriculados por año:

In [26]:
matriculados_por_año_df = semestres_df.iloc[:, 29:]
matriculados_por_año_df

,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013
0,451,433,576,684,774,756,752,783,755,745,771,779,640
1,697,675,742,715,746,684,637,615,602,613,719,633,615
2,521,528,559,579,622,602,591,589,484,520,564,508,468
3,272,235,287,382,461,444,430,504,488,499,551,572,536
4,1783,1721,1768,1590,1571,1519,1515,1516,1486,1493,1714,1651,1628
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3185,0,0,0,0,0,0,0,0,0,0,0,0,68
3186,0,0,232,394,248,0,84,0,0,0,0,0,0
3187,0,0,94,44,124,80,0,0,0,0,0,0,0
3188,0,0,138,218,460,534,370,238,100,0,0,0,0


In [27]:
matriculados_por_año_df.shape

(3190, 13)

Concatenar los dataframes que contienen las columnas qde interés:

In [28]:
to_concat_ls = [filtered1_df, matriculados_por_año_df]
matriculados_df = pd.concat(to_concat_ls, axis=1)
matriculados_df

,INSTITUCION DE EDUCACION SUPERIOR (IES),NIVEL DE FORMACION,NUCLEO BASICO DEL CONOCIMIENTO (NBC),2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013
0,UNIVERSIDAD NACIONAL DE COLOMBIA,UNIVERSITARIA,"INGENIERIA AGRONOMICA, PECUARIA Y AFINES",451,433,576,684,774,756,752,783,755,745,771,779,640
1,UNIVERSIDAD NACIONAL DE COLOMBIA,UNIVERSITARIA,ECONOMIA,697,675,742,715,746,684,637,615,602,613,719,633,615
2,UNIVERSIDAD NACIONAL DE COLOMBIA,UNIVERSITARIA,"GEOGRAFIA, HISTORIA",521,528,559,579,622,602,591,589,484,520,564,508,468
3,UNIVERSIDAD NACIONAL DE COLOMBIA,UNIVERSITARIA,"INGENIERIA AGRICOLA, FORESTAL Y AFINES",272,235,287,382,461,444,430,504,488,499,551,572,536
4,UNIVERSIDAD NACIONAL DE COLOMBIA,UNIVERSITARIA,INGENIERIA CIVIL Y AFINES,1783,1721,1768,1590,1571,1519,1515,1516,1486,1493,1714,1651,1628
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3185,CORPORACION UNIVERSITARIA DE CIENCIA Y TECNOLO...,UNIVERSITARIA,DERECHO Y AFINES,0,0,0,0,0,0,0,0,0,0,0,0,68
3186,SERVICIO NACIONAL DE APRENDIZAJE-SENA-,TECNICA PROFESIONAL,DERECHO Y AFINES,0,0,232,394,248,0,84,0,0,0,0,0,0
3187,SERVICIO NACIONAL DE APRENDIZAJE-SENA-,TECNOLOGICA,"INGENIERIA ELECTRONICA, TELECOMUNICACIONES Y A...",0,0,94,44,124,80,0,0,0,0,0,0,0
3188,SERVICIO NACIONAL DE APRENDIZAJE-SENA-,TECNOLOGICA,INGENIERIA MECANICA Y AFINES,0,0,138,218,460,534,370,238,100,0,0,0,0


In [29]:
matriculados_df["NUCLEO BASICO DEL CONOCIMIENTO (NBC)"].value_counts()

ADMINISTRACION                                            850
EDUCACION                                                 325
INGENIERIA DE SISTEMAS, TELEMATICA Y AFINES               303
CONTADURIA PUBLICA                                        275
DERECHO Y AFINES                                          149
MEDICINA                                                  124
PSICOLOGIA                                                 81
AGRONOMIA                                                  69
INGENIERIA INDUSTRIAL Y AFINES                             69
ECONOMIA                                                   65
COMUNICACION SOCIAL, PERIODISMO Y AFINES                   62
SALUD PUBLICA                                              53
INGENIERIA ELECTRONICA, TELECOMUNICACIONES Y AFINES        49
INGENIERIA AMBIENTAL, SANITARIA Y AFINES                   47
DISEÑO                                                     47
INGENIERIA MECANICA Y AFINES                               47
INGENIER

In [30]:
matriculados_df["NIVEL DE FORMACION"].value_counts()

UNIVERSITARIA          1099
ESPECIALIZACION         878
TECNOLOGICA             780
MAESTRIA                219
TECNICA PROFESIONAL     155
DOCTORADO                59
Name: NIVEL DE FORMACION, dtype: int64

In [31]:
matriculados_df["INSTITUCION DE EDUCACION SUPERIOR (IES)"].value_counts()

CORPORACION UNIVERSITARIA REMINGTON                                    690
UNIVERSIDAD DE ANTIOQUIA                                               468
TECNOLOGICO DE ANTIOQUIA                                               329
UNIVERSIDAD PONTIFICIA BOLIVARIANA                                     248
FUNDACION UNIVERSITARIA LUIS AMIGO FUNLAM                              192
UNIVERSIDAD EAFIT-                                                     141
UNIVERSIDAD DE MEDELLIN                                                136
UNIVERSIDAD CES                                                        111
UNIVERSIDAD NACIONAL DE COLOMBIA                                       105
POLITECNICO COLOMBIANO JAIME ISAZA CADAVID                              92
UNIVERSIDAD COOPERATIVA DE COLOMBIA                                     91
UNIVERSIDAD DE SAN BUENAVENTURA                                         81
INSTITUCION UNIVERSITARIA PASCUAL BRAVO                                 53
FUNDACION UNIVERSITARIA E

In [32]:
stem_nbc_ls = [
              "INGENIERIA DE SISTEMAS, TELEMATICA Y AFINES", 
              "INGENIERIA INDUSTRIAL Y AFINES ",
              "ECONOMIA",
              "INGENIERIA ELECTRONICA, TELECOMUNICACIONES Y AFINES",
              "INGENIERIA AMBIENTAL, SANITARIA Y AFINES",             
              "INGENIERIA CIVIL Y AFINES",
              "INGENIERIA MECANICA Y AFINES",
              "BIOLOGIA, MICROBIOLOGIA Y AFINES",
              "INGENIERIA ELECTRICA Y AFINES",
              "INGENIERIA AGROINDUSTRIAL, ALIMENTOS Y AFINES",
              "QUIMICA Y AFINES",
              "OTRAS INGENIERIAS",
              "MATEMATICAS, ESTADISTICA Y AFINES",
              "INGENIERIA QUIMICA Y AFINES",
              "INGENIERIA AGRONOMICA, PECUARIA Y AFINES",
              "INGENIERIA AGRICOLA, FORESTAL Y AFINES",
              "INGENIERIA DE MINAS, METALURGIA Y AFINES",
              "INGENIERIA ADMINISTRATIVA Y AFINES",
              "BACTERIOLOGIA",
              "GEOLOGIA, OTROS PROGRAMAS DE CIENCIAS NATURALES",
              "FISICA",
              "INGENIERIA BIOMEDICA Y AFINES",
              "INGENIERIA ELECTRONICA, TELECOMUNICACIONES Y AFINES",
              "DISENIO" 
]

In [33]:
nbc_base_dict = calligraphy_unification(stem_nbc_ls)
nbc_dict = calligraphy_unification(matriculados_df["NUCLEO BASICO DEL CONOCIMIENTO (NBC)"])
ies_dict = calligraphy_unification(matriculados_df["INSTITUCION DE EDUCACION SUPERIOR (IES)"])
nf_dict = calligraphy_unification(matriculados_df["NIVEL DE FORMACION"])

In [34]:
matriculados1_df = pd.DataFrame()

In [35]:
for row_num in range(0, len(matriculados_df["INSTITUCION DE EDUCACION SUPERIOR (IES)"])):
  initial_str = matriculados_df.loc[row_num,"INSTITUCION DE EDUCACION SUPERIOR (IES)"]
  if initial_str in ies_dict.keys():
    matriculados1_df.loc[row_num, "INSTITUCION DE EDUCACION SUPERIOR (IES)"] = ies_dict[initial_str]

In [36]:
for row_num in range(0, len(matriculados_df["NIVEL DE FORMACION"])):
  initial_str = matriculados_df.loc[row_num,"NIVEL DE FORMACION"]
  if initial_str in nf_dict.keys():
    matriculados1_df.loc[row_num, "NIVEL DE FORMACION"] = nf_dict[initial_str]

In [37]:
for row_num in range(0, len(matriculados_df["NUCLEO BASICO DEL CONOCIMIENTO (NBC)"])):
  initial_str = matriculados_df.loc[row_num,"NUCLEO BASICO DEL CONOCIMIENTO (NBC)"]
  if initial_str in nbc_dict.keys():
    matriculados_df.loc[row_num, "NUCLEO BASICO DEL CONOCIMIENTO (NBC)"] = nbc_dict[initial_str]
  if matriculados_df.loc[row_num, "NUCLEO BASICO DEL CONOCIMIENTO (NBC)"] in nbc_base_dict.values():
    matriculados1_df.loc[row_num, "NUCLEO BASICO DEL CONOCIMIENTO (NBC)"] = matriculados_df.loc[row_num, "NUCLEO BASICO DEL CONOCIMIENTO (NBC)"] 
  else:
    matriculados1_df.loc[row_num, "NUCLEO BASICO DEL CONOCIMIENTO (NBC)"] = "Null"

In [38]:
for año_col in range(2001, 2014):
  matriculados1_df[str(año_col)] =  matriculados_df[str(año_col)]


In [39]:
matriculados1_df

,INSTITUCION DE EDUCACION SUPERIOR (IES),NIVEL DE FORMACION,NUCLEO BASICO DEL CONOCIMIENTO (NBC),2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013
0,UNIVERSIDAD NACIONAL DE COLOMBIA,UNIVERSITARIA,INGENIERIA AGRONOMICA PECUARIA Y AFINES,451,433,576,684,774,756,752,783,755,745,771,779,640
1,UNIVERSIDAD NACIONAL DE COLOMBIA,UNIVERSITARIA,ECONOMIA,697,675,742,715,746,684,637,615,602,613,719,633,615
2,UNIVERSIDAD NACIONAL DE COLOMBIA,UNIVERSITARIA,Null,521,528,559,579,622,602,591,589,484,520,564,508,468
3,UNIVERSIDAD NACIONAL DE COLOMBIA,UNIVERSITARIA,INGENIERIA AGRICOLA FORESTAL Y AFINES,272,235,287,382,461,444,430,504,488,499,551,572,536
4,UNIVERSIDAD NACIONAL DE COLOMBIA,UNIVERSITARIA,INGENIERIA CIVIL Y AFINES,1783,1721,1768,1590,1571,1519,1515,1516,1486,1493,1714,1651,1628
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3185,CORPORACION UNIVERSITARIA DE CIENCIA Y TECNOLO...,UNIVERSITARIA,Null,0,0,0,0,0,0,0,0,0,0,0,0,68
3186,SERVICIO NACIONAL DE APRENDIZAJESENA,TECNICA PROFESIONAL,Null,0,0,232,394,248,0,84,0,0,0,0,0,0
3187,SERVICIO NACIONAL DE APRENDIZAJESENA,TECNOLOGICA,INGENIERIA ELECTRONICA TELECOMUNICACIONES Y AF...,0,0,94,44,124,80,0,0,0,0,0,0,0
3188,SERVICIO NACIONAL DE APRENDIZAJESENA,TECNOLOGICA,INGENIERIA MECANICA Y AFINES,0,0,138,218,460,534,370,238,100,0,0,0,0


In [40]:
matriculados1_df = matriculados1_df.loc[matriculados1_df["NUCLEO BASICO DEL CONOCIMIENTO (NBC)"] != "Null", :]

In [41]:
matriculados1_df.index = list(range(0, len(matriculados1_df["INSTITUCION DE EDUCACION SUPERIOR (IES)"])))

In [42]:
matriculados1_df["NUCLEO BASICO DEL CONOCIMIENTO (NBC)"].value_counts()

INGENIERIA DE SISTEMAS TELEMATICA Y AFINES            303
ECONOMIA                                               65
INGENIERIA ELECTRONICA TELECOMUNICACIONES Y AFINES     49
INGENIERIA AMBIENTAL SANITARIA Y AFINES                47
INGENIERIA MECANICA Y AFINES                           47
INGENIERIA CIVIL Y AFINES                              40
BIOLOGIA MICROBIOLOGIA Y AFINES                        31
OTRAS INGENIERIAS                                      28
INGENIERIA ELECTRICA Y AFINES                          22
MATEMATICAS ESTADISTICA Y AFINES                       20
INGENIERIA ADMINISTRATIVA Y AFINES                     17
INGENIERIA AGROINDUSTRIAL ALIMENTOS Y AFINES           12
BACTERIOLOGIA                                          11
INGENIERIA DE MINAS METALURGIA Y AFINES                10
QUIMICA Y AFINES                                       10
INGENIERIA QUIMICA Y AFINES                            10
INGENIERIA BIOMEDICA Y AFINES                           9
GEOLOGIA OTROS

In [43]:
sum = 0
resta = 0
for valor in matriculados1_df["NUCLEO BASICO DEL CONOCIMIENTO (NBC)"]:
  if valor in nbc_base_dict.values():
    sum = sum + 1
  else:
    resta = resta + 1  
print(sum, resta)


753 0


Volver a poner las comas en la columna de NBC:

In [44]:
nbc_base_dict

{'BACTERIOLOGIA': 'BACTERIOLOGIA',
 'BIOLOGIA, MICROBIOLOGIA Y AFINES': 'BIOLOGIA MICROBIOLOGIA Y AFINES',
 'DISENIO': 'DISENIO',
 'ECONOMIA': 'ECONOMIA',
 'FISICA': 'FISICA',
 'GEOLOGIA, OTROS PROGRAMAS DE CIENCIAS NATURALES': 'GEOLOGIA OTROS PROGRAMAS DE CIENCIAS NATURALES',
 'INGENIERIA ADMINISTRATIVA Y AFINES': 'INGENIERIA ADMINISTRATIVA Y AFINES',
 'INGENIERIA AGRICOLA, FORESTAL Y AFINES': 'INGENIERIA AGRICOLA FORESTAL Y AFINES',
 'INGENIERIA AGROINDUSTRIAL, ALIMENTOS Y AFINES': 'INGENIERIA AGROINDUSTRIAL ALIMENTOS Y AFINES',
 'INGENIERIA AGRONOMICA, PECUARIA Y AFINES': 'INGENIERIA AGRONOMICA PECUARIA Y AFINES',
 'INGENIERIA AMBIENTAL, SANITARIA Y AFINES': 'INGENIERIA AMBIENTAL SANITARIA Y AFINES',
 'INGENIERIA BIOMEDICA Y AFINES': 'INGENIERIA BIOMEDICA Y AFINES',
 'INGENIERIA CIVIL Y AFINES': 'INGENIERIA CIVIL Y AFINES',
 'INGENIERIA DE MINAS, METALURGIA Y AFINES': 'INGENIERIA DE MINAS METALURGIA Y AFINES',
 'INGENIERIA DE SISTEMAS, TELEMATICA Y AFINES': 'INGENIERIA DE SISTEMAS T

In [45]:
coma_dict = dict()
for key, value in nbc_base_dict.items():
  coma_dict[value] = key

coma_dict

{'BACTERIOLOGIA': 'BACTERIOLOGIA',
 'BIOLOGIA MICROBIOLOGIA Y AFINES': 'BIOLOGIA, MICROBIOLOGIA Y AFINES',
 'DISENIO': 'DISENIO',
 'ECONOMIA': 'ECONOMIA',
 'FISICA': 'FISICA',
 'GEOLOGIA OTROS PROGRAMAS DE CIENCIAS NATURALES': 'GEOLOGIA, OTROS PROGRAMAS DE CIENCIAS NATURALES',
 'INGENIERIA ADMINISTRATIVA Y AFINES': 'INGENIERIA ADMINISTRATIVA Y AFINES',
 'INGENIERIA AGRICOLA FORESTAL Y AFINES': 'INGENIERIA AGRICOLA, FORESTAL Y AFINES',
 'INGENIERIA AGROINDUSTRIAL ALIMENTOS Y AFINES': 'INGENIERIA AGROINDUSTRIAL, ALIMENTOS Y AFINES',
 'INGENIERIA AGRONOMICA PECUARIA Y AFINES': 'INGENIERIA AGRONOMICA, PECUARIA Y AFINES',
 'INGENIERIA AMBIENTAL SANITARIA Y AFINES': 'INGENIERIA AMBIENTAL, SANITARIA Y AFINES',
 'INGENIERIA BIOMEDICA Y AFINES': 'INGENIERIA BIOMEDICA Y AFINES',
 'INGENIERIA CIVIL Y AFINES': 'INGENIERIA CIVIL Y AFINES',
 'INGENIERIA DE MINAS METALURGIA Y AFINES': 'INGENIERIA DE MINAS, METALURGIA Y AFINES',
 'INGENIERIA DE SISTEMAS TELEMATICA Y AFINES': 'INGENIERIA DE SISTEMAS, T

In [46]:
for row_num in range(0, len(matriculados1_df["NUCLEO BASICO DEL CONOCIMIENTO (NBC)"])):
  key = matriculados1_df.loc[row_num,"NUCLEO BASICO DEL CONOCIMIENTO (NBC)"]
  if key in coma_dict.keys():
    matriculados1_df.loc[row_num, "NUCLEO BASICO DEL CONOCIMIENTO (NBC)"] = coma_dict[key]
 

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [47]:
matriculados1_df["NUCLEO BASICO DEL CONOCIMIENTO (NBC)"].value_counts()

INGENIERIA DE SISTEMAS, TELEMATICA Y AFINES            303
ECONOMIA                                                65
INGENIERIA ELECTRONICA, TELECOMUNICACIONES Y AFINES     49
INGENIERIA MECANICA Y AFINES                            47
INGENIERIA AMBIENTAL, SANITARIA Y AFINES                47
INGENIERIA CIVIL Y AFINES                               40
BIOLOGIA, MICROBIOLOGIA Y AFINES                        31
OTRAS INGENIERIAS                                       28
INGENIERIA ELECTRICA Y AFINES                           22
MATEMATICAS, ESTADISTICA Y AFINES                       20
INGENIERIA ADMINISTRATIVA Y AFINES                      17
INGENIERIA AGROINDUSTRIAL, ALIMENTOS Y AFINES           12
BACTERIOLOGIA                                           11
INGENIERIA QUIMICA Y AFINES                             10
QUIMICA Y AFINES                                        10
INGENIERIA DE MINAS, METALURGIA Y AFINES                10
INGENIERIA BIOMEDICA Y AFINES                           

In [48]:
matriculados1_df

,INSTITUCION DE EDUCACION SUPERIOR (IES),NIVEL DE FORMACION,NUCLEO BASICO DEL CONOCIMIENTO (NBC),2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013
0,UNIVERSIDAD NACIONAL DE COLOMBIA,UNIVERSITARIA,"INGENIERIA AGRONOMICA, PECUARIA Y AFINES",451,433,576,684,774,756,752,783,755,745,771,779,640
1,UNIVERSIDAD NACIONAL DE COLOMBIA,UNIVERSITARIA,ECONOMIA,697,675,742,715,746,684,637,615,602,613,719,633,615
2,UNIVERSIDAD NACIONAL DE COLOMBIA,UNIVERSITARIA,"INGENIERIA AGRICOLA, FORESTAL Y AFINES",272,235,287,382,461,444,430,504,488,499,551,572,536
3,UNIVERSIDAD NACIONAL DE COLOMBIA,UNIVERSITARIA,INGENIERIA CIVIL Y AFINES,1783,1721,1768,1590,1571,1519,1515,1516,1486,1493,1714,1651,1628
4,UNIVERSIDAD NACIONAL DE COLOMBIA,UNIVERSITARIA,"INGENIERIA DE MINAS, METALURGIA Y AFINES",386,376,410,451,495,517,588,619,641,636,671,672,678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
748,CORPORACION ACADEMIA TECNOLOGICA DE COLOMBIA ATEC,TECNOLOGICA,INGENIERIA MECANICA Y AFINES,0,0,0,0,11,4,0,0,0,0,0,0,0
749,CORPORACION ACADEMIA TECNOLOGICA DE COLOMBIA ATEC,TECNOLOGICA,"INGENIERIA ELECTRONICA, TELECOMUNICACIONES Y A...",0,0,0,0,5,10,0,0,0,0,0,0,0
750,CORPORACION ACADEMIA SUPERIOR DE ARTES,TECNICA PROFESIONAL,INGENIERIA MECANICA Y AFINES,26,0,0,1,0,215,0,0,0,0,0,0,0
751,SERVICIO NACIONAL DE APRENDIZAJESENA,TECNOLOGICA,"INGENIERIA ELECTRONICA, TELECOMUNICACIONES Y A...",0,0,94,44,124,80,0,0,0,0,0,0,0


In [49]:
!mkdir /content/WTM_Medellin_Pilot/Piloto_Medellin/data/clean

In [50]:
matriculados1_df.to_csv("/content/WTM_Medellin_Pilot/Piloto_Medellin/data/clean/matriculados_hasta_2013.csv", index=False)